In [1]:
import torch
import mlx.core as mx

In [10]:
from proteus.utils import coerce_torch_to_mx

l = 2
tq = torch.randn((1, 1, l, l))
tk, tv = torch.randn_like(tq), torch.randn_like(tq)

mq, mk, mv = coerce_torch_to_mx(tq), coerce_torch_to_mx(tk), coerce_torch_to_mx(tv)

tq, tk, tv

coercing torch to mlx
coercing torch to mlx
coercing torch to mlx


(tensor([[[[ 0.061902, -0.515321],
           [ 0.013836,  0.440570]]]]),
 tensor([[[[ 0.037734, -0.143145],
           [ 1.022153, -1.524982]]]]),
 tensor([[[[ 0.087644,  0.563132],
           [-1.652426,  1.438317]]]]))

In [35]:
torch_mask = torch.ones((l, l), dtype=torch.float32).tril()
# torch_mask = torch.zeros_like(torch_mask, dtype=torch.float32).masked_fill(torch_mask.logical_not(), float('-inf'))

mlx_mask = coerce_torch_to_mx(torch_mask)

torch_mask, mlx_mask

coercing torch to mlx


(tensor([[1., 0.],
         [1., 1.]]),
 array([[1, 0],
        [1, 1]], dtype=float32))

In [36]:
from math import sqrt
torch.set_printoptions(precision=6)

scale = sqrt(tq.size(-1))**-1

torch_attn = torch.ops.aten.scaled_dot_product_attention.default
mlx_attn = mx.fast.scaled_dot_product_attention


(
    torch_attn(tq, tk, tv, attn_mask=torch_mask, scale=scale), 
    mlx_attn(mq, mk, mv, mask=mlx_mask, scale=scale)
)

(tensor([[[[-0.588470,  0.903190],
           [-0.601968,  0.909979]]]]),
 array([[[[-0.58847, 0.90319],
          [-0.601968, 0.909979]]]], dtype=float32))

when torch sdpa gets a boolean tensor mask, it fills mask with -inf where the tensor is false and 0 where true, whereas MLX fills with 1 where true and 0 where false, so the solution is convert the boolean tensor manually to the -inf tensor for MLX

In [49]:
x = mx.ones((4, 4))
mask = mx.tril(mx.ones((4, 4), dtype=mx.bool_))
mx.where(mx.logical_not(mask), -float('inf'), 0.)

mx.triu(mx.full((4, 4), -float('inf')), k=1)

array([[0, -inf, -inf, -inf],
       [0, 0, -inf, -inf],
       [0, 0, 0, -inf],
       [0, 0, 0, 0]], dtype=float32)